# Random forest classifier: diabetes prediction

Absolutely minimal MVP (minimum viable product) solution.

## 1. Data acquisition

In [1]:
import pandas as pd

# Load the data from the URL
data_df=pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/decision-tree-project-tutorial/main/diabetes.csv")

### 1.2. Train-test split

In [2]:
from sklearn.model_selection import train_test_split

# Separate features from labels
labels=data_df['Outcome']
features=data_df.drop('Outcome', axis=1)

# Split the data into training and testing features and labels
training_features, testing_features, training_labels, testing_labels=train_test_split(
    features,
    labels,
    test_size=0.2,
    random_state=315
)

In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


## 2. EDA

### 2.1. Features

In [4]:
# Inspect the training features' data types
training_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, 765 to 611
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               614 non-null    int64  
 1   Glucose                   614 non-null    int64  
 2   BloodPressure             614 non-null    int64  
 3   SkinThickness             614 non-null    int64  
 4   Insulin                   614 non-null    int64  
 5   BMI                       614 non-null    float64
 6   DiabetesPedigreeFunction  614 non-null    float64
 7   Age                       614 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 43.2 KB


All of the features are already numeric - we don't need to do anything for the model to run. Let's check the labels too.

### 2.2. Labels

In [5]:
training_labels.info()

<class 'pandas.core.series.Series'>
Index: 614 entries, 765 to 611
Series name: Outcome
Non-Null Count  Dtype
--------------  -----
614 non-null    int64
dtypes: int64(1)
memory usage: 9.6 KB


Also, already numeric - we can move right to training the model and setting a baseline performance result.

## 3. Training

In [6]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate and train decision tree classifier
model=RandomForestClassifier(random_state=315)
fit_result=model.fit(training_features, training_labels)

## 4. Evaluation

In [7]:
from sklearn.metrics import accuracy_score

# Make predictions from test set features
predicted_labels=model.predict(testing_features)

# Score predictions from accuracy
percent_accuracy=accuracy_score(testing_labels, predicted_labels) * 100
print(f'Model is {percent_accuracy:.1f}% accurate on the test data')

Model is 77.9% accurate on the test data


In [8]:
zero_counts = (training_features == 0).sum()

sorted_zero_counts = zero_counts.sort_values(ascending=False)

print("Columns with the most zeroes:")
print(sorted_zero_counts)

Columns with the most zeroes:
Insulin                     294
SkinThickness               172
Pregnancies                  93
BloodPressure                23
BMI                           7
Glucose                       4
DiabetesPedigreeFunction      0
Age                           0
dtype: int64


In [9]:
df_filtered = training_features.drop(columns=['Insulin'])

print(df_filtered)
df_filtered.describe()

     Pregnancies  Glucose  BloodPressure  SkinThickness   BMI  \
765            5      121             72             23  26.2   
74             1       79             75             30  32.0   
733            2      106             56             27  29.0   
740           11      120             80             37  42.3   
0              6      148             72             35  33.6   
..           ...      ...            ...            ...   ...   
275            2      100             70             52  40.5   
746            1      147             94             41  49.3   
194            8       85             55             20  24.4   
567            6       92             62             32  32.0   
611            3      174             58             22  32.9   

     DiabetesPedigreeFunction  Age  
765                     0.245   30  
74                      0.396   22  
733                     0.426   22  
740                     0.785   48  
0                       0.627   50

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,3.851792,120.605863,69.602606,21.307818,32.215961,0.473090,33.245928
std,3.403173,31.483407,18.224136,16.055309,7.706636,0.339908,11.742608
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000
25%,1.000000,100.000000,64.000000,0.000000,27.500000,0.239250,24.000000
50%,3.000000,117.000000,72.000000,24.000000,32.350000,0.370500,29.000000
75%,6.000000,138.000000,80.000000,33.000000,36.575000,0.628500,41.000000
max,17.000000,199.000000,122.000000,99.000000,67.100000,2.420000,72.000000


Ok, done! Absolutely minimal random forest classifier using ~10 statements. Out of the box, the random forest performs slightly better than a single decision tree classifier. But, there are still many things we could do to try and improve it.

In [10]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier


param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2', None]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)

grid_search.fit(training_features, training_labels)

best_model = grid_search.best_estimator_

cv_scores = cross_val_score(best_model, training_features, training_labels, cv=5)

print(f"Cross-validation score: {cv_scores.mean() * 100:.2f}%")

predicted_labels = best_model.predict(testing_features)

percent_accuracy = accuracy_score(testing_labels, predicted_labels) * 100
print(f'Model is {percent_accuracy:.1f}% accurate on the test data')

print(f"Best hyperparameters: {grid_search.best_params_}")

print(f"Best cross-validation score during grid search: {grid_search.best_score_:.4f}")


/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
540 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
264 fits failed with the following error:
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/vscode/.l

Cross-validation score: 77.37%
Model is 76.0% accurate on the test data
Best hyperparameters: {'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Best cross-validation score during grid search: 0.7737


In [11]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 8],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 5],
    'max_features': ['auto', 'sqrt', 'log2', None],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)

grid_search.fit(training_features, training_labels)

best_model = grid_search.best_estimator_

cv_scores = cross_val_score(best_model, training_features, training_labels, cv=5)

print(f"Cross-validation score: {cv_scores.mean() * 100:.2f}%")

predicted_labels = best_model.predict(testing_features)

percent_accuracy = accuracy_score(testing_labels, predicted_labels) * 100
print(f'Model is {percent_accuracy:.1f}% accurate on the test data')

print(f"Best hyperparameters: {grid_search.best_params_}")

print(f"Best cross-validation score during grid search: {grid_search.best_score_:.4f}")


/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
480 fits failed out of a total of 1920.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
254 fits failed with the following error:
Traceback (most recent call last):
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/vscode/.local/lib/python3.10/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/home/vscode/.l

Cross-validation score: 77.20%
Model is 77.3% accurate on the test data
Best hyperparameters: {'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Best cross-validation score during grid search: 0.7720


In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300, 500, 1000],   
    'max_depth': [None, 10, 20, 30, 40],           
    'min_samples_split': [2, 5, 10, 20, 30],      
    'min_samples_leaf': [1, 2, 4, 5, 10],          
    'max_features': ['auto', 'sqrt', 'log2', None, 0.3, 0.5], 
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)

grid_search.fit(training_features, training_labels)

best_model = grid_search.best_estimator_

cv_scores = cross_val_score(best_model, training_features, training_labels, cv=5)

print(f"Cross-validation score: {cv_scores.mean() * 100:.2f}%")

predicted_labels = best_model.predict(testing_features)

percent_accuracy = accuracy_score(testing_labels, predicted_labels) * 100
print(f'Model is {percent_accuracy:.1f}% accurate on the test data')

print(f"Best hyperparameters: {grid_search.best_params_}")

print(f"Best cross-validation score during grid search: {grid_search.best_score_:.4f}")

/home/vscode/.local/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
